# ==========================================
# 📊 Trading Prediction App (Streamlit)
# ==========================================
# Predict next-day stock price movement (UP/DOWN)
# using RandomForestClassifier and Yahoo Finance data.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# ----------------------------
# 🧱 Sidebar Inputs
# ----------------------------

In [2]:
st.sidebar.title("📈 Stock Predictor")
asset = st.sidebar.selectbox("Select Asset", ["AAPL", "GOOG", "MSFT", "TSLA", "BTC-USD", "ETH-USD"])
period = st.sidebar.selectbox("Select Time Frame", ["1y", "2y", "5y"])

2025-11-03 20:58:05.562 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 20:58:06.855 
  command:

    streamlit run c:\Users\jasva\Desktop\Mine\AI\Trading_Prediction\cloud\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-03 20:58:06.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 20:58:06.858 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 20:58:06.858 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 20:58:06.859 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 20:58:06.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 20:58

# ----------------------------
# 📥 Fetch Data
# ----------------------------

In [3]:
st.write(f"### Fetching data for **{asset}** ({period})...")
data = yf.download(asset, period=period, interval="1d")

data["Return"] = data["Close"].pct_change()
data["MA5"] = data["Close"].rolling(5).mean()
data["MA10"] = data["Close"].rolling(10).mean()
data["MA20"] = data["Close"].rolling(20).mean()
data["Target"] = np.where(data["Close"].shift(-1) > data["Close"], 1, 0)
data.dropna(inplace=True)

2025-11-03 20:58:06.886 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 20:58:06.888 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-03 20:58:06.888 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
C:\Users\jasva\AppData\Local\Temp\ipykernel_26452\3387209164.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(asset, period=period, interval="1d")
[*********************100%***********************]  1 of 1 completed


# ----------------------------
# 🧠 Train Model
# ----------------------------

In [ ]:
features = ["Return", "MA5", "MA10", "MA20"]
X = data[features]
y = data["Target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# ----------------------------
# 📊 Results
# ----------------------------

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
st.write(f"### ✅ Model Accuracy: **{accuracy*100:.2f}%**")

# ----------------------------
# 🧩 Plot Predictions (fixed)
# ----------------------------

In [ ]:
data_test = data.iloc[len(X_train):].copy()
data_test = data_test.iloc[:len(y_pred)]  # ensure same length
data_test["Predicted"] = y_pred

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(data_test.index, data_test["Close"], label="Actual Price")

# Only plot predicted buy signals
buy_signals = data_test[data_test["Predicted"] == 1]
ax.scatter(
    buy_signals.index,
    buy_signals["Close"],
    label="Predicted Buy Signal",
    color="green",
    marker="^",
    alpha=0.8
)

ax.set_title(f"{asset} — Predicted Trading Signals")
ax.legend()
st.pyplot(fig)

# ----------------------------
# 🔮 Predict Next Day
# ----------------------------

In [ ]:
latest = data[features].iloc[-1:].values
next_day = model.predict(latest)[0]

if next_day == 1:
    st.success("📈 The model predicts: Price will go **UP** tomorrow.")
else:
    st.error("📉 The model predicts: Price will go **DOWN** tomorrow.")